<a href="https://colab.research.google.com/github/Fulankeee/Credit-Risk-Early-Warning-Project/blob/main/notebooks/01_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>